## 参考坤哥的简书 详见： https://www.jianshu.com/p/d2d0ff48f7d6

## 导入相关的库

In [2]:
from bs4 import BeautifulSoup as bs
import requests


In [37]:
resopnes=requests.get("http://www.worldrowing.com/events/2018-world-rowing-under-23-championships/u23-mens-eight/")
soup=bs(resopnes.text,"html.parser")

In [23]:
athelte_list=soup.select("tr.resultsDetails li")#通过观察移动鼠标时，
                  #在网页中框定的内容范围来推测，提取出哪些标签中的内容。
athelte_list[:1]

[<li>
 <figure class="listWrap" data-odfid="46136">
 <div class="listThumb">
 <a href="/athletes/athlete/46136/wertz-brennan">
 <img alt="WERTZ, Brennan" class="basicShadow" src="/assets/images/ml/athletes/46136.jpg"/>
 </a>
 </div>
 <figcaption>
 <h4><a href="/athletes/athlete/46136/wertz-brennan" title="WERTZ, Brennan">WERTZ, Brennan</a></h4>
 <p class="yPadding">
 																			Position: Bow
 																		</p>
 </figcaption>
 </figure>
 </li>]

In [10]:
#找到运动员姓名所在的标签 。原文写得好 ：“通过观察网页html文件查看字段对应标签和标签的层次结构，
#我们就可以开始编写代码实现我们的爬虫。”
athelte_one=athelte_list[0].select('h4 a')[0]  #"h4 a"就是名字所在的标签层次结构
athelte_one

<a href="/athletes/athlete/46136/wertz-brennan" title="WERTZ, Brennan">WERTZ, Brennan</a>

In [14]:
#.text获取运动员的名字。
athelte_one_name=athelte_list[0].select('h4 a')[0].text
athelte_one_name

'WERTZ, Brennan'

In [19]:
athelte_one_place=athelte_list[0].select('p.yPadding')[0]
athelte_one_place

<p class="yPadding">
																			Position: Bow
																		</p>

In [21]:
#打印出text
athelte_one_text=athelte_list[0].select('p.yPadding')[0].text
athelte_one_text

'\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tPosition: Bow\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t'

In [22]:
#去掉position两边的"\r","\t",得到运动员的为位置
athelte_one_PLACE=athelte_list[0].select('p.yPadding')[0].text.strip()
athelte_one_PLACE

'Position: Bow'

In [29]:
athelte_one_img_url=athelte_list[0].select('img')[0]["src"]
athelte_one_img_url

'/assets/images/ml/athletes/46136.jpg'

In [32]:
#获取到每个运动员照片的链接。注意链接由两部分组成，前一部分是"http://www.worldrowing.com"，
#另一半是爬取到的，两者平到一起即为最终的链接
athelte_one_IMG_URL="http://www.worldrowing.com"+athelte_list[0].select('img')[0]["src"]
athelte_one_IMG_URL

'http://www.worldrowing.com/assets/images/ml/athletes/46136.jpg'

## 所以获取网页中的运动员姓名，位置，和照片的连接的代码就很简单了

In [33]:
#获取千把个运动的相关信息
for athelte in athelte_list[:8]:
    ATHELTE_NAME=athelte.select("h4 a")[0].text
    ATHELTE_POSITION=athelte.select("p.yPadding")[0].text.strip()
    ATHELTE_IMG_URL="http://www.worldrowing.com"+athelte.select("img")[0]["src"]
    print(ATHELTE_NAME,ATHELTE_POSITION,ATHELTE_IMG_URL)


WERTZ, Brennan Position: Bow http://www.worldrowing.com/assets/images/ml/athletes/46136.jpg
BEST, Justin Position: Seat 2 http://www.worldrowing.com/assets/images/ml/athletes/47042.jpg
CARLSON, Christopher Position: Seat 3 http://www.worldrowing.com/assets/images/ml/athletes/49843.jpg
LANDBOE, Arne Position: Seat 4 http://www.worldrowing.com/assets/images/ml/athletes/48679.jpg
MOLITOR, Madison Position: Seat 5 http://www.worldrowing.com/assets/images/ml/athletes/51534.jpg
HALBERT, Samuel Position: Seat 6 http://www.worldrowing.com/assets/images/ml/athletes/51532.jpg
GRADY, Michael Position: Seat 7 http://www.worldrowing.com/assets/images/ml/athletes/44646.jpg
GAARD, Andrew Position: Stroke http://www.worldrowing.com/assets/images/ml/athletes/46141.jpg


In [ ]:
#类似的。获取更详细的sex,country,birthday的信息，这些信息在更详细的链接里面。
#并把所有获取的信息以元素是字典的大列表，储存在DataFrame中。

In [52]:
#完整代码
#完整代码
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests

response = requests.get('http://www.worldrowing.com/events/2018-world-rowing-under-23-championships/u23-mens-eight/')
soup = bs(response.text, 'html.parser')
athelte_list=soup.select("tr.resultsDetails li")
item_list=[]
for athelte in athelte_list:
    item={}
    item["name"]=athelte.select("h4 a")[0].text
    item["position"]=athelte.select("p.yPadding")[0].text.strip()
    item["img_url"]="http://www.worldrowing.com"+athelte.select("img")[0]["src"]
    detail_url="http://www.worldrowing.com"+athelte.select("h4 a")[0]["href"]
    response=requests.get(detail_url)
    soup=bs(response.text,"html.parser")
    item["sex"]=soup.select("div.dd")[0].text
    item["country"]=soup.select("h1.athleteInfoTitle span")[0].text
    item["birthday"]=soup.select("div.dd")[1].text #看下面就知道为啥是[1]了
    item_list.append(item)
    
df=pd.DataFrame(item_list,columns=item_list[0].keys())
print(item_list[0].keys())
df.to_excel("athleteINfo.xlsx")   #还可以这样，有点骚！
    
#在select里面填上啥，还是不咋明白。 要找啥，就在网页上选中，再审查元素，找到。



dict_keys(['name', 'position', 'img_url', 'sex', 'country', 'birthday'])


In [48]:
respones=requests.get("http://www.worldrowing.com/athletes/athlete/46136/wertz-brennan")
soup=bs(respones.text,"html.parser")
sex=soup.select('div.dd')   #是个列表，有两个元素，第一个储存了性别，第二个储存了birthday
sex

[<div class="dd" itemprop="gender">M</div>, <div class="dd">13 May 1997</div>]